In [ ]:
# Root - Coefficients Dataset Generation Code 
# For each 6 coefficients of the quintic equation having integer spacing from the range (-5,5) , 5 roots are found which can be taken as a training data for NN.

import numpy as np
import itertools
import csv
import time
import os

def find_polynomial_roots(coeff_range=(-5,5), max_combinations=None, output_file=r"C:\Users\Akshay Patil\Desktop\Roots\R.csv", 
                         batch_size=10000, save_interval=100000):
    """
    Find roots for degree 5 polynomials with varying coefficients from -5 to 5
    
    Parameters:
    coeff_range: tuple of (min, max) coefficient values
    max_combinations: maximum number of combinations to process (None for all)
    output_file: file to save results
    batch_size: number of polynomials to process at once
    save_interval: save to CSV after processing this many polynomials
    """
    
    # Generate coefficient range
    coeffs = list(range(coeff_range[0], coeff_range[1] + 1))
    total_combinations = len(coeffs)**6
    
    print(f"Finding roots for degree 5 polynomials with coefficients from {coeff_range[0]} to {coeff_range[1]}")
    print(f"Total possible combinations: {total_combinations}")
    
    if max_combinations:
        print(f"Will process up to {max_combinations} combinations")
    
    # Create CSV file and write header
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['polynomial_id', 'a', 'b', 'c', 'd', 'e', 'f', 'polynomial_equation', 
                     'root_1', 'root_2', 'root_3', 'root_4', 'root_5']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
    
    # Generate all combinations
    all_combinations = itertools.product(coeffs, repeat=6)
    
    processed = 0
    valid_count = 0
    skipped_count = 0
    start_time = time.time()
    batch_results = []
    
    # Process combinations
    for poly_id, coeffs in enumerate(all_combinations, 1):
        a, b, c, d, e, f = coeffs
        
        # Skip if leading coefficient is 0 (not a degree 5 polynomial)
        if a == 0:
            skipped_count += 1
            continue
            
        try:
            # Find roots using numpy
            # Coefficients in descending order: [a, b, c, d, e, f]
            roots = np.roots([a, b, c, d, e, f])
            
            # Store result
            result = {
                'polynomial_id': valid_count + 1,
                'a': a, 'b': b, 'c': c, 'd': d, 'e': e, 'f': f,
                'polynomial_equation': f"{a}*x^5 + {b}*x^4 + {c}*x^3 + {d}*x^2 + {e}*x + {f}",
                'root_1': '', 'root_2': '', 'root_3': '', 'root_4': '', 'root_5': ''
            }
            
            # Add roots
            for i, root in enumerate(roots):
                result[f'root_{i+1}'] = str(root)
            
            batch_results.append(result)
            valid_count += 1
            
        except Exception as e:
            # Skip errors
            skipped_count += 1
            continue
        
        processed += 1
        
        # Save batch to CSV when it reaches batch_size or save_interval
        if len(batch_results) >= batch_size or processed % save_interval == 0 or processed == max_combinations:
            with open(output_file, 'a', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerows(batch_results)
            
            # Clear batch
            batch_results = []
            
            # Print progress
            elapsed = time.time() - start_time
            rate = processed / elapsed if elapsed > 0 else 0
            remaining = (total_combinations - processed) / rate if rate > 0 else 0
            
            print(f"Processed: {processed:,}/{total_combinations:,} ({processed/total_combinations*100:.2f}%)")
            print(f"Valid polynomials: {valid_count:,}, Skipped: {skipped_count:,}")
            print(f"Time elapsed: {elapsed:.2f}s, Rate: {rate:.2f} polynomials/s")
            print(f"Estimated time remaining: {remaining/60:.2f} minutes")
            print("-" * 50)
        
        # Check if we've reached the maximum
        if max_combinations and processed >= max_combinations:
            break
    
    # Save any remaining results
    if batch_results:
        with open(output_file, 'a', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerows(batch_results)
    
    # Final statistics
    total_time = time.time() - start_time
    print(f"\nProcessing complete!")
    print(f"Total processed: {processed:,} polynomials")
    print(f"Valid polynomials: {valid_count:,}")
    print(f"Skipped polynomials: {skipped_count:,}")
    print(f"Total time: {total_time:.2f}s")
    print(f"Results saved to: {output_file}")

# Run the function to find all polynomial roots
# To process all 1.77M combinations:
# find_polynomial_roots(coeff_range=(-5, 5))

# To process exactly 1 million combinations:
find_polynomial_roots(coeff_range=(-5, 5), max_combinations=None)